In [ ]:
%matplotlib inline
import matplotlib
import seaborn as sns
matplotlib.rcParams['savefig.dpi'] = 144

# Creating Spark Applications
<!-- requirement: projects/simple-spark-project -->
<!-- requirement: projects/simple-pyspark-project -->
<!-- requirement: small_data/gutenberg -->

## REPLs

Spark has a built-in **R**ead-**E**valuate-**P**rint-**L**oop in the form of a shell that you can use for interactive analysis (similar to ipython).

*Scala*: `$SPARK_HOME/bin/spark-shell`

*Python*: `$SPARK_HOME/bin/pyspark`

You can also use Spark kernels in Jupyter notebooks.

*Scala*: The Spark-Scala kernel automatically puts `val sc` in the namespace as a handle to the Spark Context. 

*Python*: Simply `import pyspark`, then eg.
`sc = pyspark.SparkContext("local[*]", "demo")`.

If this doesn't work, make sure your PYTHONPATH contains the module:
``` bash
export PYTHONPATH=$SPARK_HOME/python:$SPARK_HOME/python/lib/:$PYTHONPATH
```

*R*: Same thing:
```
library(SparkR, lib.loc = c(file.path(Sys.getenv("SPARK_HOME"), "R", "lib")))

sparkR.session(master = "local[*]", sparkConfig = list(spark.driver.memory = "2g"))
```

In [1]:
# Telling Spark to look in the local file system
import os
def localpath(path):
    return 'file://' + str(os.path.abspath(os.path.curdir)) + '/' + path

In [2]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext

conf = SparkConf()
conf.setMaster("local[*]")
conf.setAppName("main")
sc = SparkContext(conf=conf)
sqlContext = SQLContext(sc)

lines = sc.textFile(localpath("small_data/gutenberg/"))
totalLines = lines.count()
print "total lines: %d" % totalLines

sc.stop()

total lines: 77931


### Notes

* You can only have one Spark Context active at a time. You'll need to stop it before starting a new one.
* Scala (like Java) requires everything to be inside a class or object.  The repl will accept a global assignment like
```scala
val x = 1
```
but this will not work in application code that you compile.  You would need to put this into an object or class.

* The PySpark API is similar to Scala, but not exactly, and there may be missing features. Be careful when looking through the [documentation](https://spark.apache.org/docs/latest/programming-guide.html).
* PySpark comes installed with all versions of Spark, and you should be able to `import pyspark` without any trouble.
* For machine learning applications, you'll probably need to work within a Spark SQL context as well as the usual Spark Context - this is to enable DataFrame functionality.

## Building Spark applications

While the interactive console is fun, it is (likely) not how you will be submitting a job.  Instead, you will want to follow these steps.

### Python

It's straightforward to run a PySpark script as an app. Simply pass it to spark-submit:

`spark-submit --py-files src/classes.py src/app.py arg1 arg2`

In this example we pass command line arguments to main.py using `argparse` and eg. `def main(arg1, arg2):`

Use the `--py-files` flag with `spark-submit` to specify additional Python modules which should be made available to each worker. These may include class definitions or third-party dependencies. Usually, if you're using classes, you will not be able to define them in the main file.

If you have many python modules that need to be distributed to the worker nodes, they can be combined in zip file, which can be passed as a `--py-files` argument.

A sample simple application is provided in [projects/simple-pyspark-project](projects/simple-pyspark-project).  The main script is in [SimpleApp.py](projects/simple-pyspark-project/SimpleApp.py).  It imports a utility function from [myutils.py](projects/simple-pyspark-project/myutils.py), so it can be launched as so:

```bash
$ spark-submit --master local[*] --py-files myutils.py \
  SimpleApp.py <path on HDFS>
```

### Scala

The equivalent Scala version of this sample application is in [projects/simple-spark-project](projects/simple-spark-project).  There is now an additional build step.

1. **Build your Spark application**: Scala is a compiled language so you will need to build a jar that can be run on the Java Virtual Machine (JVM).  JAR (Java Archive) is a package file format typically used to aggregate many Java class files and associated metadata and resources (text, images, etc.) into one file to distribute application software or libraries on the Java platform.  Go to the project directory `projects/simple-spark-project` and run
```bash
$ sbt package
```

2. Submit the job locally running on 4 cores:
```bash
$ spark-submit \
     --class "com.thedataincubator.simplespark.SimpleApp" \
     --master local[4] \
     target/scala-2.10/simple-project_2.10-1.0.jar <path on HDFS>
```

You can use `local[*]` to run with as many worker threads as logical cores on your machine.

## Packaging using sbt

### What is `sbt`?
`sbt` is a modern build tool written in and for Scala, though it is also a general purpose build tool.  `sbt` is actually a Scala [Domain Specific Language (DSL)](https://en.wikipedia.org/wiki/Domain-specific_language), meaning it's actually Scala (with enough new constructs to look like it's not Scala).  To invoke SBT, run
``` bash
$ sbt
```
which brings you into an "sbt session."  The commands given below are to be typed within an SBT session.

### Why `sbt`?
- Sane(ish) dependency management
- Incremental recompilation and keeping the compiler alive in between compilations (see [this article](http://www.scala-sbt.org/0.13.2/docs/Detailed-Topics/Understanding-incremental-recompilation.html))
- Automatic recompilation triggered by file-change.  Within an sbt session, enter:
    ```
    ~compile
    ```
- Run the program within sbt:
    ```
    run
    ```
- Test the program within sbt:
    ```
    test
    ```
- Full Scala language support for creating tasks (it's a DSL)
- Launch REPL in project context
    ```
    console # gives you a Scala repl within your jar
    ```
    and you can type commands into the REPL to play around
    ```scala
    import com.thedataincubator.simplespark.SimpleApp

    val x = 1
    ```

## Project layout and directory structure

A sample simple application is provided in [projects/simple-spark-project](projects/simple-spark-project).

### Source files:
1. `src/main` – your app code goes here, in a subdirectory indicating the code’s language, e.g.
    1. `src/main/scala`
    1. `src/main/java`
1. `src/main/resources` – static files you want added to your jar (e.g. logging config)
1. `src/test` – like src/main, but for tests  
1. `src/main/scala/com/thedataincubator/simplespark/SimpleApp.scala` - an actual code file.  This is in two components:
    1. `src/main/scala` - overhead (explained above)
    1. `com/thedataincubator/simplespark/` - related to the package hierarchy (b/c it's written by people at the domain `thedataincubator.com`).  There are two files in our sample app:
        1. `src/main/scala/com/thedataincubator/simplespark/SimpleApp.scala` - main app
        1. `src/main/scala/com/thedataincubator/simplespark/Foo.scala` - helper class and methods
    
    This affects your code in two places:
        1. All your `*.scala` files in this directory need to declare their packages consistently with their directory
        ```scala
        package com.thedataincubator.simplespark
        ```
        You can then easily access other files in this folder (package).  For example, there is also a `Foo.scala` in this directory (with the same package definition) and we can access it directly in `SimpleApp.scala` 
        1. When you invoke the jar, you need to specify the class (package) name (see the `spark-submit` command above)

### Build files:
1. `build.sbt` - This is like a make file.  It tells `sbt` how to build your project.  You can specify the version of your application, the Scala version you want, and the version of the dependencies you require (e.g. Spark) in the `build.sbt`:
    ```scala
    name := "Simple Project"
    version := "1.0"
    scalaVersion := "2.10.4"
    libraryDependencies += "org.apache.spark" %% "spark-core" % "1.2.0"
    ```
1. `project/` – Because the `sbt` compiler is actually Scala code, the compiler has to be built.  It is built with `sbt`.  The instructions for how to build this meta-build are placed in this directory.  This allows you to tweak the build's build.
1. `project/build.sbt` - The instructions for the meta-build (like `build.sbt` but for the compiler, not for your main project).  You can also tweak the build's build's build by having `project/project` and continue iterating forever (see [Organizing Build's](http://www.scala-sbt.org/0.13/tutorial/Organizing-Build.html)).

### Output files:
1. `target/` – The destination for generated files (e.g. class files, jars).

### Further information:
For more, check out the [documentation](http://www.scala-sbt.org/0.13/tutorial/Directories.html).

### Alternatives:
[Maven](https://maven.apache.org/) is another project management tool known for its reporting, documentation, and compatibility with continuous integration. SBT has become more popular as Scala becomes more ubiquitous.

## Spark on Amazon Web Services

Amazon Web Services has extremely thorough documentation around everything from the commands available to the command line interface (CLI) `aws {commands}`, to the Python wrapper for said interface `boto`, to full tutorials and examples on how to fire up an EMR cluster or a bunch of EC2 instances with almost any desired data processing framework.

EC2 is cheaper than EMR (Elastic Map Reduce), but EMR is recommended for immediate use of Hadoop and any other project in the ecosystem because it takes care of the Hadoop/YARN configuration for us. It's somewhat configurable via [Amazon Machine Images](http://docs.aws.amazon.com/ElasticMapReduce/latest/DeveloperGuide/ami-versions-supported.html) (AMIs). In a production setting it's possible you'll want to use specific versions for consistency; in our case it's safe to use the most recent version (`3.8.0` at the time of this writing).

You can use the boto API in Python to programmatically check the status of jobs and clusters.

### Setting up a personal AWS account

To use AWS you'll need to [create an account](http://aws.amazon.com/) if you haven't already. For the first year after new account creation, you'll be eligible for discounts on some services as part of the Free Tier program.

Access the AWS [web console](https://console.aws.amazon.com/s3/) to handle most of your configuration. You'll need at least one S3 bucket to serve as storage for your logs and output. You'll also want to upload your actual code.

From there you can create EMR clusters as you wish and run jobs. Be careful about the nodes you use, as only certain sizes are eligible for the free tier discounts. Still, you only pay for what you use, and the costs for small, educational jobs are relatively manageable.

There's an in depth [tutorial](http://docs.aws.amazon.com/ElasticMapReduce/latest/DeveloperGuide/emr-get-started.html) available, and more detailed cluster configuration information can be found in this notebook, and in the Spark module.

Also note that in addition to the normal credentials you might need to take care of:
* Generating an EC2 keypair (this is separate from the AWS general keypair and goes in .mrjob.conf)
* `aws emr create-default-roles` if you plan on just using the defaults for EMR
* Making sure your user is part of a group with sufficient permissions (admin is probably fine)
* You can set up multiple profiles in the `~/.aws/credentials` file in order to facilitate copying data from our S3 bucket while still being able to access your own.

### AWS credentials and command line tools

1. To verify that it is working, try 
``` bash
aws s3 ls
```
You should get back a json blob that contains the permissions you just added for your user.  If not, double-check that you got your permissions setup correctly.

1. `boto` ([docs](https://boto.readthedocs.org/en/latest/)) is a python library that wraps the functionality of `awscli`.  You can install it using
``` bash
pip install boto
```
and follow the instructions in the docs to get started.

1. Another option for interacting with s3 from the command line is `s3cmd`. You can download/start using it via   
``` bash
git clone https://github.com/s3tools/s3cmd.git
```
and follow the documentation [here](https://github.com/s3tools/s3cmd).

### S3 buckets

You will need to create one (or more) AWS S3 buckets.  These will hold both the input data and the source code or JAR files for your application.  They will also provide a location for writing logs and output from your application.  Buckets can be managed from the [web console](https://console.aws.amazon.com/s3/home).

For our purposes, we will assume that you've created the bucket `s3://my-bucket/`.  In here you will create directories `src/`, where you upload your code or JAR file, and `logs/`, where EMR will write logs from the cluster.  Often you will also create `data/` to store the input data, and `output/` to store the processed data.

### Third party software

There are some third-party tools that can help navigate AWS S3. It can be time-consuming to go through the command line looking for logs when there's no autocomplete or easily viewable directory structure - in which case something like Bucket Explorer might save you some time.

### Running jobs

Before running a job, we need to create a cluster to run the job on.  This can be done via the [web console](https://console.aws.amazon.com/elasticmapreduce/home) or from the command line.  The following command will launch a cluster of three *m3.xlarge* machines.  One will act as the master node, while the other two will be slave nodes running the computation.  (This should be entered as a single line without the comments; we've added comments and removed line-continuation characters for clarity.)

```bash
aws emr create-cluster
  --release-label emr-5.4.0        # Sets software versions
  --name 'MySparkCluster'
  --log-uri 's3://my-bucket/logs`
  --use-default-roles
  --ec2-attributes KeyName=my-key  # Name of SSH keypair
  --applications Name=Spark        # Install Spark appropriately
  --instance-type m3.xlarge
  --instance-count 3
```

This will immediately return a cluster name, like *j-3VIY0CQC7QOFN*, but it will take several minutes for the cluster to come up fully.  You can watch the progress on the web console.  Jobs can be added to the cluster with the `add-steps` command:

*Scala/Java*:
```bash
aws emr add-steps
  --cluster-id j-3VIY0CQC7QOFN     # From creation step
  --steps Name=SimpleApp,Type=spark,Args=[--deploy-mode,cluster,--master,yarn,--class,com.my-org.my-app.Main,s3://my-bucket/src/Application.jar,arg1,arg2,...],ActionOnFailure=CONTINUE
```

*Python*:
```bash
aws emr add-steps
  --cluster-id j-3VIY0CQC7QOFN     # From creation step
  --steps Name=SimpleApp,Type=spark,Args=[--deploy-mode,cluster,--master,yarn,--py-files,s3://my-bucket/src/utilites.py,s3://my-bucket/src/main.py,arg1,arg2,...],ActionOnFailure=CONTINUE
```

Note that the arguments in the `Args` section are exactly those passed to `spark-submit`.  Here, `arg1`, `arg2`, etc., represent the arguments to be passed to your application.  These will generally include both an S3 bucket for data input and an S3 bucket to store the output.

The cluster will continue to run (and cost you money!) when the job is complete.  Don't forget to shut it down from the web console or with

```bash
aws emr terminate-clusters --cluster-id j-3VIY0CQC7QOFN
```

This work flow is handy, in that it allows you to examine the cluster to try to figure out why a job failed.  Once you have the kinks ironed out, it can be handy to launch a cluster, immediately run a job on it, and shut it back down.  This can be accomplished by passing the `--steps` argument to `create-cluster` along with the `--auto-terminate` flag.

There are a number of other options to play with.  Generally you need more compute power on the worker nodes than on the master nodes.  This command will create a cluster of 1 *m3.xlarge* machine as master and 9 *m3.2xlarge* machines as slaves.  It will immediate run a particular JAR file and then shut down when finished.


``` bash
aws emr create-cluster
  --release-label emr-5.4.0
  --name 'MySparkCluster'
  --log-uri 's3://my-bucket/logs`
  --use-default-roles
  --ec2-attributes KeyName=my-key
  --applications Name=Spark
  --instance-groups Name=Master,InstanceGroupType=MASTER,InstanceType=m3.xlarge,InstanceCount=1 Name=Core,InstanceGroupType=CORE,InstanceType=m3.2xlarge,InstanceCount=9
  --auto-terminate
  --steps Name=SparkApp,Type=spark,
Args=[--deploy-mode,cluster,--master,yarn,--class,com.my-org.my-app.Main,s3://my-bucket/src/Application.jar,arg1,arg2,...],ActionOnFailure=CONTINUE
```

Note that, when using third-party libraries, it is preferable to install them at bootstrapping time. See the [documentation](http://docs.aws.amazon.com/ElasticMapReduce/latest/DeveloperGuide/emr-plan-bootstrap.html) for examples of how to write a script that will run at cluster creation.

### Spark on EC2

Spark comes with several built-in scripts to launch and manage clusters on Amazon EC2. Data input/output will be a little trickier than EMR but there are some scripts which can help with that too. Check out the [documentation](http://spark.apache.org/docs/1.6.2/ec2-scripts.html).

## Spark on Google Cloud Platform

Cloud Dataproc is GCP's analog to EMR: a managed Hadoop cluster environment that uses Google Compute Engine instances under the hood. There's a comprehensive 60-day free trial with $300 of credit (which should be more than enough for our purposes). Here are some step-by-step instructions for getting started:

1. Probably the easiest way to interact with GCP is through the [Cloud SDK](https://cloud.google.com/sdk/#Quick_Start). Once you authenticate through your Google Account, you won't need any other credentials and most of the necessary command line tools will be there.
1. Make buckets on Google Storage as necessary.
1. Go into the API manager and enable both the GCE and Dataproc APIs.
1. `gsutil cp` to upload data to your buckets. Useful flags are -r (recursive) and -m (parallel operation for many small tasks). eg. `gsutil -m cp -r data/stackoverflow gs://mybucket/data/stackoverflow/`
1. Scala apps have the dependencies packaged in. If you're using Python, you'll need to write a bash script that runs on cluster initialization if you need third party libraries:
``` bash
#!/bin/bash
apt-get install -y python-pip
pip install toolz
pip install lxml
```
Save this as init.sh and upload it to gs as well.
1. You will likely want to disable dynamic executor allocation due to the way Dataproc does cluster management.
1. Creating a cluster:
``` bash
gcloud dataproc clusters create cluster-1 --initialization-actions gs://mybucket/code/init.sh --zone us-east1-c --master-machine-type n1-standard-2 --master-boot-disk-size 50 --num-workers 3 --worker-machine-type n1-standard-2 --worker-boot-disk-size 50 --num-worker-local-ssds 1 --project google-project-123456
```
1. Submitting a job:
``` bash
gcloud dataproc jobs submit pyspark --cluster cluster-1 --py-files gs://mybucket/code/classes.py gs://mybucket/code/app.py gs://mybucket/data/input/ gs://mybucket/results/
```
1. Make sure to delete your cluster after the job is done:
``` bash
gcloud dataproc clusters delete cluster-1`
```
Most of this can also be done from the web console. There's a [good tutorial](https://cloud.google.com/dataproc/tutorials/spark-scala) for Spark with Scala as well.

### Upgrading from the free trial

The free trial limits you to 8 YARN cores, including the master node, which realistically means the biggest cluster you can use on dataproc has 3 worker instances with 2 nodes each.

It is a significant time saver to upgrade to a paid account. You will keep your free trial credit (it expires when your trial would have expired), the only difference is **you will have to manually cancel your account** to avoid being billed after the trial period expires. Doing this is painless and will increase your quota to 24 YARN cores.

## A digression: Plotting (in-memory) with Spark and Scala

### Adding  external dependencies
TL;DR use the `--packages` flag for external dependencies with Maven coordinates and `--jars` flag for local dependency jars for anything *not* in your `build.sbt`.

- **spark-shell**: 3rd party (Maven) dependencies in spark-shell with Maven coordinates: 
```bash
$ spark-shell --master {host} --packages "{group}:{artifact}:{version}" --jars oneJar.jar,anotherJar.jar"
```

- **spark-submit**: Include 3rd party library jars to spark-submit. Note that packages with Maven coordinates should properly be in your `build.sbt` if you are using `spark-submit`:
```bash
$ spark-submit --master {host} --jars {someJar}.jar,{anotherJar}.jar --class MyApp path/to/MyApp.jar
```


### Example: plot values in spark-shell

**N.B.** To get plots to appear from DigitalOcean on your local machine, you have to turn on X11 forwarding by ssh'ing into your box with `ssh -X`

- To add dependencies in spark-shell with Maven coordinates: 
```bash
$ spark-shell --master local[4] --packages "{group}:{artifact}:{version}"
```


1. Fire up the spark-shell:
```bash
$ spark-shell --master local[*] --packages "org.scalanlp:breeze-viz_2.10:0.11.2"
```

1. Create a plot that appears in a separate window

```scala
scala> val a = Array(0.3, 0.9, 1.4, 34.21, -3.4)
a: Array[Double] = Array(0.3, 0.9, 1.4, 34.21, -3.4)

scala> import breeze.plot._
import breeze.plot._

scala> val f = Figure()
f: breeze.plot.Figure = breeze.plot.Figure@73543048

scala> val p = f.subplot(0)
p: breeze.plot.Plot = breeze.plot.Plot@2e549515

scala> p += plot(a,a,'.')
res4: breeze.plot.Plot = breeze.plot.Plot@2e549515

scala> val g = breeze.stats.distributions.Gaussian(0,1)
...
scala> val p2 = f.subplot(1,1)
...
scala> p2 += hist(g.sample(100000),100)
```

### Example: plot a distribution

Check out Cloudera's [`KernelDensity.estimate` implementation](https://github.com/sryza/aas/tree/master/ch09-risk/src/main/scala/com/cloudera/datascience/risk)

```scala
import breeze.plot._
import com.cloudera.datascience.risk._

object AnalysisClass {
    ...
  def plotDistribution(samples: Array[Double]): Figure = {
    val min = samples.min
    val max = samples.max
    // Using toList before toArray avoids a Scala bug
    val domain = Range.Double(min, max, (max - min) / 100).toList.toArray
    val densities = KernelDensity.estimate(samples, domain)
    val f = Figure()
    val p = f.subplot(0)
    p += plot(domain, densities)
    p.ylabel = "Density"
    f
  }
}
```

In order to run this, include the *breeze* dependency in your `build.sbt`, and include the path to the `com.cloudera.datascience` package in the `--jars` flag of spark-submit.

```scala
libraryDependencies  ++= Seq(
  "org.scalanlp" %% "breeze" % "0.11.2",
  "org.scalanlp" %% "breeze-natives" % "0.11.2"
)
```

*Copyright &copy; 2015 The Data Incubator.  All rights reserved.*